In [ ]:
import os
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Data Processing
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.metrics import roc_curve, roc_auc_score
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer
# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import warnings
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
import warnings


In [ ]:

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
# Chemin vers le répertoire contenant les fichiers CSV
dir_path = "ref"

In [ ]:
# Liste des noms de fichiers CSV dans le répertoire
file_names = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
print(file_names)

In [ ]:
# Liste des DataFrames chargés à partir des fichiers CSV
dfs = []
for file_name in file_names:
    file_path = os.path.join(dir_path, file_name)
    df = pd.read_csv(file_path)
    df = df.drop(df.columns[[0,-2,-3]], axis=1)
    dfs.append(df)

In [ ]:
# Concaténer les DataFrames en un seul DataFrame
df_concat = pd.concat(dfs)

In [ ]:
# Afficher les 5 premières lignes du DataFrame concaténé
print(df_concat.head())
print(df_concat.columns)
print(df_concat)

In [ ]:
# Utiliser la méthode isna() pour vérifier chaque cellule du dataframe
nan_df = df_concat.isna()

# Utiliser la méthode sum() pour compter le nombre total de valeurs NaN
nb_nan = nan_df.sum().sum()

print(nb_nan)  # Affiche 3

In [ ]:
# Sélectionner les colonnes à utiliser pour l'arbre de décision
feature_cols = list(df_concat.columns[:-1])

In [ ]:
# Sélectionner la colonne cible pour l'arbre de décision
target_col = df_concat.columns[-1]

In [ ]:
# Diviser le DataFrame en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_concat[feature_cols], df_concat[target_col], test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape)

In [ ]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier

scorer = make_scorer(f1_score, average = 'weighted')

In [ ]:
# create the grid
#n_estimators = [10, 100,200]
n_estimators=[100]
#max_depth = [2,3,4]
#max_depth = [20]
grid_space={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }


#params = {'max_features': ['sqrt', 'log2', None],'n_estimators': [25, 50, 100, 150],'max_depth': [3, 6, 9],'max_leaf_nodes': [3, 6, 9]}
params = {"n_estimators":[100], "max_depth":[10], 'max_features':[7],'min_samples_leaf':[1],'min_samples_split':[2]}

In [ ]:
# instantiate the classifier 

rfc = yClassifier(random_state=0)

In [ ]:
rfc_cv = GridSearchCV(rfc,params, scoring=scorer, n_jobs=-1, verbose=1, cv=2)

In [ ]:

rfc_cv.fit(X_train, y_train)

In [ ]:
print(rfc_cv.best_estimator_)
print(f"Best parameters: {rfc_cv.best_params_}")
print(rfc_cv.best_score_)

In [ ]:
# # fit the model
best_params = rfc_cv.best_params_
best_rf= RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)




In [ ]:
class_names = df_concat[target_col].unique()
print(class_names)

In [ ]:
# # Predict the Test set results

y_pred = best_rf.predict(X_test)
print(y_pred)
print(classification_report(y_pred, y_test))

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# Create a confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cnf_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))


disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix,display_labels=class_names)
disp.plot(ax=ax, xticks_rotation=50, values_format='d')



plt.show()

print(classification_report(y_test, y_pred))



In [ ]:
print_score(best_rf, X_train, y_train, X_test, y_test, train=True)
print_score(best_rf, X_train, y_train, X_test, y_test, train=False)

In [ ]:
# Créer un objet graphviz pour visualiser l'arbre de décision
#dot_data = export_graphviz(best_rf, out_file=None, feature_names=feature_cols, filled=True, rounded=True, special_characters=True,class_names =class_names)
dot_data = export_graphviz(best_rf.estimators_[0], out_file=None, feature_names=feature_cols,
                           class_names=class_names, filled=True, rounded=True,
                           special_characters=True)

In [ ]:
# Transformer le fichier .dot en une image représentant l'arbre de décision

display(dot_data)

In [ ]:
graph = graphviz.Source(dot_data)
graph

In [ ]:
# Sauvegarder le graphique dans un fichier PNG
#graph.render(filename='random_forest_tree', format='png')

In [ ]:
# feature importance
feature_scores = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

print(feature_scores)

In [ ]:
f_i = list(zip(X_train.columns,best_rf.feature_importances_))
f_i.sort(key = lambda x : x[1])
plt.barh([x[0] for x in f_i],[x[1] for x in f_i])

plt.show()

In [ ]:
# permutation feature
perm_importance = permutation_importance(best_rf, X_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()


In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_train.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
# Cross validate model with Kfold stratified cross val
from sklearn.model_selection import StratifiedKFold
K_fold = StratifiedKFold(n_splits=10)

In [ ]:

from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import learning_curve    

scores = cross_val_score(best_rf, X_train, y_train, cv=K_fold, 
                        n_jobs=4, scoring='accuracy')

print(scores)
round(np.mean(scores)*100, 2)

In [ ]:
y_score = best_rf.predict_proba(X_test)
print(y_score)
print('roc_auc_score for Random forest: ', roc_auc_score(y_test, y_score,multi_class='ovr'))

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
        
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
plot_learning_curve(estimator = best_rf ,title = "Random forest - Learning Curve",
                    X = X_train, y = y_train, cv = K_fold);

In [ ]:
X=df_concat[feature_cols]
y=df_concat[target_col]
random_state = np.random.RandomState(0)

In [ ]:
n_samples, n_features = X.shape
n_classes = len(np.unique(y))
print(f"{n_classes} classes")


In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
print(y_onehot_test.shape)  # (n_samples, n_classes)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_onehot_test.ravel(),
    y_score.ravel(),
    name="micro-average OvR",
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Micro-averaged One-vs-Rest\nReceiver Operating Characteristic")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

micro_roc_auc_ovr = roc_auc_score(
    y_test,
    y_score,
    multi_class="ovr",
    average="micro",
)

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")

In [ ]:
macro_roc_auc_ovr = roc_auc_score(
    y_test,
    y_score,
    multi_class="ovr",
    average="macro",
)

print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{macro_roc_auc_ovr:.2f}")

In [ ]:
from sklearn.metrics import roc_curve, auc

# store the fpr, tpr, and roc_auc for all averaging strategies
fpr, tpr, roc_auc = dict(), dict(), dict()
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{roc_auc['micro']:.2f}")

In [ ]:
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_onehot_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr_grid = np.linspace(0.0, 1.0, 1000)

# Interpolate all ROC curves at these points
mean_tpr = np.zeros_like(fpr_grid)

for i in range(n_classes):
    mean_tpr += np.interp(fpr_grid, fpr[i], tpr[i])  # linear interpolation

# Average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = fpr_grid
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc['macro']:.2f}")

In [ ]:
from itertools import cycle

fig, ax = plt.subplots(figsize=(6, 6))

plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.2f})",
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label=f"macro-average ROC curve (AUC = {roc_auc['macro']:.2f})",
    color="navy",
    linestyle=":",
    linewidth=4,
)

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for class_id, color in zip(range(n_classes), colors):
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_score[:, class_id],
        name=f"ROC curve for {class_names[class_id]}",
        color=color,
        ax=ax,
    )

plt.plot([0, 1], [0, 1], "k--", label="ROC curve for chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Extension of Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()